# AIモデル訓練サンプル
PythonでEng-Fraサンプルデータを用いて，EnglishからFrenchへ翻訳するモデルを訓練する。


## プログラム説明

model.py

内部で，pytorchのtransformerライブラリを呼び出している。各パラメタの詳細は [Pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) を参照のこと。

- emb_size    
  単語のエンベッドサイズ
- nhead    
  transformerのMultiHeadAttentionのヘッド数
- ffn_hid_dim       
  FeedForwardNeuralNetworkの次元数
- batch_size     
  ミニバッチサイズ。メモリが足りないときや計算速度を早めたいときにはこのサイズを変更する
- num_encoder_layers    
  エンコーダ内のサブエンコーダ層の数 
- num_decoder_layers    
  デコーダ内のサブデコーダ層の数
- lr   
  学習率
- dropout    
  ドロップアウトの割合，1=100%
- num_epochs    
  学習用データを何周学習するか
- device    
  cuda: Cudaが使えるマシンではこれを選択
  mps: Apple Silicornが使えるマシンではこれを選択
  cpu: CPUで計算
- earlystop_patient    
  num_epochs以下でも，開発用データで，Lossが下がらなくなった回数がearlystop_patientより大きくなると，計算を終了させる
- output_dir    
  学習したモデルを格納するディレクトリ。ディレクトリには checkpoint_xxx.pt（xxxはepoch数）とcheckpoint_best.ptが作成され，valid lossが最も小さくなったepoch回のモデルをcheckpoint_best.ptとして保存する
- tensorboard_logdir    
  tensorboard のログを格納するディレクトリ。学習結果などを視覚化して表示できる。 tensorboard --logdir tensorboard_logdir で起動し，http://localhost:6006でアクセスすると表示される
- prefix    
  jsonl形式の訓練データ及び開発データのprefix
- source_lang    
  jsonl形式の訓練データ及び開発データでの，翻訳元となるデータにつけるキー
- target_lang     
  jsonl形式の訓練データ及び開発データでの，翻訳先となるデータにつけるキー

In [ ]:
# Pythonで訓練をする
# deviceはcuda or cpu or mps
#  mps: apple silicon
# out of memoryが発生した際には、batch_sizeを減らす


# 途中から計算するときには，modelディレクトリに checkpoint_xxx.pt(xxxは計算済みのepoch数)とcheckpoint_best.pt が存在すること

!python ./model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 1024 \
  --batch_size 64 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device mps \
  --earlystop_patient 3 \
  --output_dir model \
  --tensorboard_logdir logs \
  --prefix translation \
  --source_lang eng \
  --target_lang fra \
  --train_file ../dataset/train.jsonl \
  --valid_file ../dataset/val.jsonl

[nltk_data] Downloading package punkt_tab to /Users/utsubo-
[nltk_data]     katsuhiko/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/utsubo-
[nltk_data]     katsuhiko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
num_epochs:100
Epoch: 1, Train loss: 7.208, Val loss: 6.330, Epoch time = 424.666s


In [3]:
# javaのDJLで使えるようにモデルファイルを変換する

In [ ]:
!python convert.py  \
    --model_file=model/checkpoint_best.pt \
    --model_script=model/script.pt \
    --encoder=model/encoder.pt \
    --decoder=model/decoder.pt \
    --positional_encoding=model/positional_encoding.pt \
    --generator=model/generator.pt \
    --src_tok_emb=model/src_tok_emb.pt \
    --tgt_tok_emb=model/tgt_tok_emb.pt \
    --vocab_src=model/vocab_src.txt \
    --vocab_tgt=model/vocab_tgt.txt \
    --params=model/params.json \
    --device=cpu
